In [227]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

from spacepy import pycdf
import spiceypy
import os.path
import glob
import urllib.request
import json

# import spacepy.toolbox as tb
# tb.update(leapsecs=True)

#Plotly imports
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'browser'

#direct to where Solar Orbiter data functions are
import sys
sys.path.insert(1, '/Volumes/External/Computer_Backup_20230207/Python Scripts/SC_DATA')
import functions_solo as solo_data
import functions_sta_beacon as sta_data
import functions_noaa as dscovr_data

import positions_solo as solo_pos
import positions_sta as sta_pos

import coord_transforms as transform

# Load in Solar Orbiter data and positions

### Available functions for MAG include:
##### Internal files: get_solomag_range_formagonly_internal
e.g. solo_L2_mag-rtn-ll-internal_20230225_V00.cdf
##### For MAG only files: get_solomag_range_formagonly
e.g. solo_L2_mag-rtn-normal-formagonly_20200415_V01.cdf
##### For MAG only 1 minute res files: get_solomag_range_formagonly_1min
e.g. solo_L2_mag-rtn-normal-1-minute-formagonly_20200419_V01.cdf
##### For public data, 1 second resolution: get_solomag_range_1sec
e.g. solo_L2_mag-rtn-normal_20211006.cdf
##### For public data, 1 minute resolution: get_solomag_range_1min
e.g. solo_L2_mag-rtn-normal-1-minute_20200415.cdf

In [228]:
#input start and end time of data received
start_timestamp = datetime(2023, 3, 21)
end_timestamp = datetime(2023, 3, 23)

In [229]:
#load in data between timestamps provided, returns DataFrame
#may need to include path='data location' in function
solo_mag = solo_data.get_solomag_range_formagonly_internal(start_timestamp, end_timestamp).reset_index(drop=True)

In [230]:
#get spacecraft positions in HEEQ from timestamps of mag data
#function may need updated kernels
solo_pos = solo_pos.get_solo_positions(solo_mag['timestamp'])

In [231]:
#join both dataframes together at timestamp, preserving index
solo_df = solo_mag.join(solo_pos.set_index('timestamp'), on='timestamp')

In [232]:
t_now_date = datetime.utcnow().strftime("%Y-%m-%d")
#pickle dataframe
solo_df.to_pickle(f"./solo_df_{t_now_date}.pkl")

In [233]:
# clear kernels from loading positions (otherwise new kernels for other missions cannot be loaded below)
spiceypy.kclear()

In [234]:
#unpickle dataframe
# solo_df = pd.read_pickle("./solo_df.pkl")

# Plot Solar Orbiter MAG Data

In [235]:
t_start = solo_df['timestamp'][0]
t_end = solo_df['timestamp'][solo_df.shape[0]-1]

t_now = datetime.utcnow()
lag = (t_now-t_end)/timedelta(hours=1)

r_recent = solo_df['r'][solo_df.shape[0]-1]
lat_recent = solo_df['lat'][solo_df.shape[0]-1]
lon_recent = solo_df['lon'][solo_df.shape[0]-1]

## Create with Plotly (for interactive/ selection of boundaries)

In [236]:
nrows = 1
fig = make_subplots(rows=nrows, cols=1, shared_xaxes=True)

for column, color in zip(['b_x', 'b_y', 'b_z', 'b_tot'], ['red', 'green', 'blue', 'black']):
    fig.add_trace(
        go.Scatter(
            x=solo_df['timestamp'],
            y=solo_df[column],
            name=column.upper(),
            line_color=color
        ),
        row=1, col=1
    )

fig.update_yaxes(title_text='B [nT]', row=1, col=1)

fig.update_yaxes(showgrid=True, zeroline=False, showticklabels=True,
                     showspikes=True, spikemode='across', spikesnap='cursor', showline=False, spikedash='solid', spikethickness=1)
fig.update_xaxes(showgrid=True, zeroline=False, showticklabels=True, rangeslider_visible=False,
                     showspikes=True, spikemode='across', spikesnap='cursor', showline=False, spikedash='solid', spikethickness=1)

fig.update_layout(
    title=f'SolO MAG Data from {t_start} to {t_end} <br>Latest SolO Position: r = {r_recent:.2f} AU, lat = {lat_recent:.2f}, lon = {lon_recent:.2f} <br>Time lag to current time: {lag:.2f} hours')

fig.write_html(f'/Users/emmadavies/Documents/ASWO/RealTime_Preparation/SolO_MAG_{t_now_date}.html')

fig.show()


# NOAA Realtime Data (Past 7 days)

##### Calling directly from json file: json files weren't producing same data as seen on the realtime plots at https://www.swpc.noaa.gov/products/real-time-solar-wind until Mar 21, so download file manually and load using 'get_noaa_realtime_alt()' if this is the case 

In [259]:
# noaa_mag_gsm = dscovr_data.get_noaa_realtime_alt()
noaa_mag_gsm = dscovr_data.get_noaa_mag_realtime_7days()

In [260]:
#transform from GSM to RTN approx
noaa_mag = transform.GSM_to_RTN_approx(noaa_mag_gsm)

In [261]:
noaa_plas = dscovr_data.get_noaa_plas_realtime_7days()

In [262]:
noaa_df = noaa_mag.join(noaa_plas.set_index('timestamp'), on='timestamp')
noaa_df['timestamp'] = pd.to_datetime(noaa_df['timestamp'])

t_now_date_hour = datetime.utcnow().strftime("%Y-%m-%d_%H")

#pickle dataframe
noaa_df.to_pickle(f"./noaa_df_{t_now_date_hour}.pkl")

In [263]:
#unpickle dataframe
# noaa_df = pd.read_pickle("./noaa_df.pkl")

In [264]:
t_start_noaa = noaa_df['timestamp'][0]
t_end_noaa = noaa_df['timestamp'][noaa_df.shape[0]-1]

t_now = datetime.utcnow()
lag_noaa = (t_now-t_end_noaa)/timedelta(hours=1)

In [265]:
nrows = 4

fig = make_subplots(rows=nrows, cols=1, shared_xaxes=True)

#MAG
for column, color in zip(['b_x', 'b_y', 'b_z', 'b_tot'], ['red', 'green', 'blue', 'black']):
    fig.add_trace(
        go.Scatter(
            x=noaa_df['timestamp'],
            y=noaa_df[column],
            name=column.upper(),
            line_color=color
        ),
        row=1, col=1
    )


#SWE
for column, color in zip(['v_bulk'], ['black']):
    fig.add_trace(
        go.Scatter(
            x=noaa_df['timestamp'],
            y=noaa_df[column],
            name=column.upper(),
            line_color=color
        ),
        row=2, col=1
    )

for column, color in zip(['temperature'], ['#562170']):
    fig.add_trace(
        go.Scatter(
            x=noaa_df['timestamp'],
            y=noaa_df[column],
            name=column.upper(),
            line_color=color
        ),
        row=3, col=1
    )

for column, color in zip(['density'], ['#562170']):
    fig.add_trace(
        go.Scatter(
            x=noaa_df['timestamp'],
            y=noaa_df[column],
            name=column.upper(),
            line_color=color
        ),
        row=4, col=1
    )

fig.update_yaxes(title_text='B [nT] RTN', row=1, col=1)
fig.update_yaxes(title_text='V [km/s]', row=2, col=1)
fig.update_yaxes(title_text='temp', row=3, col=1)
fig.update_yaxes(title_text='density', row=4, col=1)

fig.update_yaxes(showgrid=True, zeroline=False, showticklabels=True,
                     showspikes=True, spikemode='across', spikesnap='cursor', showline=False, spikedash='solid', spikethickness=1)
fig.update_xaxes(showgrid=True, zeroline=False, showticklabels=True, rangeslider_visible=False,
                     showspikes=True, spikemode='across', spikesnap='cursor', showline=False, spikedash='solid', spikethickness=1)

fig.add_vline(x=t_now)

fig.update_layout(
    title=f'NOAA Realtime Data for {t_start_noaa} to {t_end_noaa} <br>Time lag to current time: {lag_noaa:.2f} hours')
#<br>Latest STA Position: r = {r_recent_sta:.2f} AU, lat = {lat_recent_sta:.2f}, lon = {lon_recent_sta:.2f}

fig.write_html(f'/Users/emmadavies/Documents/ASWO/RealTime_Preparation/NOAA_MAG_PLAS_{t_now_date_hour}.html')


### STEREO A Beacon Data (Past 7 days)

##### download codes will work but seem to download corrupt files: may need to manually download cdfs everyday
##### mag: https://spdf.gsfc.nasa.gov/pub/data/stereo/ahead/beacon/2023
##### plas: https://spdf.gsfc.nasa.gov/pub/data/stereo/ahead/beacon_plastic/2023

In [54]:
# download_sta_beacon_mag()
# download_sta_beacon_plas()

In [244]:
sta_plas = sta_data.get_sta_beacon_plas_7days().reset_index(drop=True)
sta_mag = sta_data.get_sta_beacon_mag_7days().reset_index(drop=True)

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [245]:
#resample data to 1 minute cadence
rdf_plas_sta = sta_plas.set_index('timestamp')[['density', 'temperature', 'v_x', 'v_y', 'v_z', 'v_bulk']].resample('1min').mean().reset_index()
rdf_mag_sta = sta_mag.set_index('timestamp')[['b_x', 'b_y', 'b_z', 'b_tot']].resample('1min').mean().reset_index()

In [246]:
#get positions from timestamps of resampled 1min mag data
sta_positions = sta_pos.get_sta_positions(rdf_mag_sta['timestamp'])

In [247]:
#join mag, plas and position dataframes together at timestamp, preserving index
sta_df = rdf_mag_sta.join(rdf_plas_sta.set_index('timestamp'), on='timestamp').join(sta_positions.set_index('timestamp'), on='timestamp')

In [248]:
t_now_date = datetime.utcnow().strftime("%Y-%m-%d")
#pickle dataframe
sta_df.to_pickle(f"./sta_df_{t_now_date}.pkl")

In [249]:
#clear spice kernels
spiceypy.kclear()

In [250]:
#unpickle dataframe
# sta_df = pd.read_pickle("./sta_df.pkl")

In [251]:
t_start_sta = sta_mag['timestamp'][0]
t_end_sta = sta_mag['timestamp'][sta_mag.shape[0]-1]

t_now = datetime.utcnow()
lag_sta = (t_now-t_end_sta)/timedelta(hours=1)

r_recent_sta = sta_positions['r'][sta_positions.shape[0]-1]
lat_recent_sta = sta_positions['lat'][sta_positions.shape[0]-1]
lon_recent_sta = sta_positions['lon'][sta_positions.shape[0]-1]

In [252]:
nrows = 4

fig = make_subplots(rows=nrows, cols=1, shared_xaxes=True)

#STA MAG
for column, color in zip(['b_x', 'b_y', 'b_z', 'b_tot'], ['red', 'green', 'blue', 'black']):
    fig.add_trace(
        go.Scatter(
            x=sta_df['timestamp'],
            y=sta_df[column],
            name=column.upper(),
            line_color=color
        ),
        row=1, col=1
    )


#SWE
for column, color in zip(['v_bulk', 'v_x', 'v_y', 'v_z'], ['black', 'red', 'green', 'blue']):
    fig.add_trace(
        go.Scatter(
            x=sta_df['timestamp'],
            y=sta_df[column],
            name=column.upper(),
            line_color=color
        ),
        row=2, col=1
    )

for column, color in zip(['temperature'], ['#562170']):
    fig.add_trace(
        go.Scatter(
            x=sta_df['timestamp'],
            y=sta_df[column],
            name=column.upper(),
            line_color=color
        ),
        row=3, col=1
    )

for column, color in zip(['density'], ['#562170']):
    fig.add_trace(
        go.Scatter(
            x=sta_df['timestamp'],
            y=sta_df[column],
            name=column.upper(),
            line_color=color
        ),
        row=4, col=1
    )

fig.update_yaxes(title_text='B [nT]', row=1, col=1)
fig.update_yaxes(title_text='V [km/s]', row=2, col=1)
fig.update_yaxes(title_text='temp', row=3, col=1)
fig.update_yaxes(title_text='density', row=4, col=1)

fig.update_yaxes(showgrid=True, zeroline=False, showticklabels=True,
                     showspikes=True, spikemode='across', spikesnap='cursor', showline=False, spikedash='solid', spikethickness=1)
fig.update_xaxes(showgrid=True, zeroline=False, showticklabels=True, rangeslider_visible=False,
                     showspikes=True, spikemode='across', spikesnap='cursor', showline=False, spikedash='solid', spikethickness=1)

fig.add_vline(x=t_now)

fig.update_layout(
    title=f'STA Beacon Data for {t_start_sta} to {t_end_sta} <br>Latest STA Position: r = {r_recent_sta:.2f} AU, lat = {lat_recent_sta:.2f}, lon = {lon_recent_sta:.2f}<br>Time lag to current time: {lag_sta:.2f} hours')

fig.write_html(f'/Users/emmadavies/Documents/ASWO/RealTime_Preparation/STA_MAG_PLAS_{t_now_date}.html')


# If pkl files have been made, can use just these files from here

In [266]:
# unpickle dataframes
solo_df = pd.read_pickle(f"./solo_df_{t_now_date}.pkl")
noaa_df = pd.read_pickle(f"./noaa_df_{t_now_date_hour}.pkl")
sta_df = pd.read_pickle(f"./sta_df_{t_now_date}.pkl")

## Time Shift and Scale SolO data

In [267]:
solo_df_ts = solo_df.copy(deep=True)

#solo positions (most recent)
r_solo = solo_df['r'][solo_df.shape[0]-1]
lat_solo = solo_df['lat'][solo_df.shape[0]-1]
lon_solo = solo_df['lon'][solo_df.shape[0]-1]

#Earth radial dist
re = 0.992854
r_sep = re-r_solo

print(f'Distance SolO to Earth = {r_sep:.2f} AU')

v_cme = 450
print(f'Constant speed {v_cme} kms/from SolO to Earth')

au = 1.495978707E11 #divide from au to metres

t_delay=r_sep*au/(v_cme*1e3)/3600  #m, m/s, convert seconds to hours
print(f'Time Delay = {t_delay:.2f} hours')

solo_df_ts['timestamp'] = solo_df['timestamp']+timedelta(hours=t_delay)

#leitner scaling relationship for B field strength
solo_df_ts['b_tot'] = solo_df['b_tot']*(re/r_solo)**(-1.64)
solo_df_ts['b_x'] = solo_df['b_x']*(re/r_solo)**(-1.64)
solo_df_ts['b_y'] = solo_df['b_y']*(re/r_solo)**(-1.64)
solo_df_ts['b_z'] = solo_df['b_z']*(re/r_solo)**(-1.64)
#lower bound
solo_df_ts['b_tot_lb'] = solo_df['b_tot']*(re/r_solo)**(-2)
solo_df_ts['b_x_lb'] = solo_df['b_x']*(re/r_solo)**(-2)
solo_df_ts['b_y_lb'] = solo_df['b_y']*(re/r_solo)**(-2)
solo_df_ts['b_z_lb'] = solo_df['b_z']*(re/r_solo)**(-2)
#upper bound
solo_df_ts['b_tot_ub'] = solo_df['b_tot']*(re/r_solo)**(-1)
solo_df_ts['b_x_ub'] = solo_df['b_x']*(re/r_solo)**(-1)
solo_df_ts['b_y_ub'] = solo_df['b_y']*(re/r_solo)**(-1)
solo_df_ts['b_z_ub'] = solo_df['b_z']*(re/r_solo)**(-1)

Distance SolO to Earth = 0.52 AU
Constant speed 450 kms/from SolO to Earth
Time Delay = 47.74 hours


### Boundary propagation (radial, simple), error window

In [268]:
def timeshift_boundary(datetime, df, lower_speed, upper_speed):
    idx = df.set_index('timestamp').index.get_loc(datetime, method='nearest')
    r = df['r'].iloc[idx]
    lat = df['lat'].iloc[idx]
    lon = df['lon'].iloc[idx]
    t = df['timestamp'].iloc[idx]
    
    t_ts = df['timestamp'].iloc[idx]+timedelta(hours=t_delay)
    
    re = 0.992854
    au = 1.495978707E11 #divide from au to metres
    
    r_sep = re-r

    t_delay_lb = (r_sep)*au/(upper_speed*1e3)/3600  #m, m/s, convert seconds to hours
    t_delay_ub = (r_sep)*au/(lower_speed*1e3)/3600
    
    t_ts_ub = df['timestamp'].iloc[idx]+timedelta(hours=t_delay_ub)
    t_ts_lb = df['timestamp'].iloc[idx]+timedelta(hours=t_delay_lb)
    
    return t, t_ts_lb, t_ts_ub

#### Define SolO boundaries

In [269]:
# t_shock_est = datetime(2023, 2, 26, 20, 34, 29)
t_le_est = datetime(2023, 3, 21, 13, 25, 31)
t_te_est = datetime(2023, 3, 21, 19, 28, 3)

In [270]:
# t_shock, t_shock_ts_lb, t_shock_ts_ub = timeshift_boundary(t_shock_est, solo_df, 300, 700)
t_le, t_le_ts_lb, t_le_ts_ub = timeshift_boundary(t_le_est, solo_df, 400, 500)
t_te, t_te_ts_lb, t_te_ts_ub = timeshift_boundary(t_te_est, solo_df, 400, 500)

## Plot all MAG Data and SolO Prediction

In [271]:
shock_colour = "#BD88E8"
le_colour = "#FF8300"
te_colour = "#00D4FF"

nrows = 5

fig = make_subplots(rows=nrows, cols=1, shared_xaxes=True)

#SolO original MAG
for column, color in zip(['b_x', 'b_y', 'b_z', 'b_tot'], ['red', 'green', 'blue', 'black']):
    fig.add_trace(
        go.Scatter(
            x=solo_df['timestamp'],
            y=solo_df[column],
            name=column.upper(),
            line_color=color,
            showlegend=False,
        ),
        row=1, col=1
    )
#     fig.add_vline(x=t_shock, row=1, col=1, line_width=1, line_dash="dash", line_color=shock_colour)
    fig.add_vline(x=t_le, row=1, col=1, line_width=1, line_dash="dash", line_color=le_colour)
    fig.add_vline(x=t_te, row=1, col=1, line_width=1, line_dash="dash", line_color=te_colour)
    fig.layout.xaxis.showticklabels=False

#SolO Timeshifted and Scaled MAG
for column, color in zip(['b_x', 'b_y', 'b_z', 'b_tot'], ['red', 'green', 'blue', 'black']):
    fig.add_trace(
        go.Scatter(
            x=solo_df_ts['timestamp'],
            y=solo_df_ts[column],
            name=column.upper(),
            line_color=color,
            showlegend=False
        ),
        row=2, col=1
    )
#     fig.add_vrect(x0=t_shock_ts_lb, x1=t_shock_ts_ub, row=2, col=1, line_width=0, fillcolor=shock_colour, opacity=0.1)
    fig.add_vrect(x0=t_le_ts_lb, x1=t_le_ts_ub, row=2, col=1, line_width=0, fillcolor=le_colour, opacity=0.1)
    fig.add_vrect(x0=t_te_ts_lb, x1=t_te_ts_ub, row=2, col=1, line_width=0, fillcolor=te_colour, opacity=0.1)
    
#NOAA Realtime DSCOVR MAG
for column, color in zip(['b_x', 'b_y', 'b_z', 'b_tot'], ['red', 'green', 'blue', 'black']):
    fig.add_trace(
        go.Scatter(
            x=noaa_df['timestamp'],
            y=noaa_df[column],
            name=column.upper(),
            line_color=color,
            showlegend=False
        ),
        row=3, col=1
    )
#     fig.add_vrect(x0=t_shock_ts_lb, x1=t_shock_ts_ub, row=3, col=1, line_width=0, fillcolor=shock_colour, opacity=0.1)
    fig.add_vrect(x0=t_le_ts_lb, x1=t_le_ts_ub, row=3, col=1, line_width=0, fillcolor=le_colour, opacity=0.1)
    fig.add_vrect(x0=t_te_ts_lb, x1=t_te_ts_ub, row=3, col=1, line_width=0, fillcolor=te_colour, opacity=0.1)

#NOAA Realtime DSCOVR PLAS
for column, color in zip(['v_bulk'], ['black']):
    fig.add_trace(
        go.Scatter(
            x=noaa_df['timestamp'],
            y=noaa_df[column],
            name=column.upper(),
            line_color=color,
            showlegend=False
        ),
        row=4, col=1
    )    
#     fig.add_vrect(x0=t_shock_ts_lb, x1=t_shock_ts_ub, row=3, col=1, line_width=0, fillcolor=shock_colour, opacity=0.1)
    fig.add_vrect(x0=t_le_ts_lb, x1=t_le_ts_ub, row=3, col=1, line_width=0, fillcolor=le_colour, opacity=0.1)
    fig.add_vrect(x0=t_te_ts_lb, x1=t_te_ts_ub, row=3, col=1, line_width=0, fillcolor=te_colour, opacity=0.1)    
    
#STA Beacon MAG
for column, color in zip(['b_x', 'b_y', 'b_z', 'b_tot'], ['red', 'green', 'blue', 'black']):
    fig.add_trace(
        go.Scatter(
            x=sta_df['timestamp'],
            y=sta_df[column],
            name=column.upper(),
            line_color=color,
            showlegend=False
        ),
        row=5, col=1
    )
#     fig.add_vrect(x0=t_shock_ts_lb, x1=t_shock_ts_ub, row=4, col=1, line_width=0, fillcolor=shock_colour, opacity=0.1)
    fig.add_vrect(x0=t_le_ts_lb, x1=t_le_ts_ub, row=4, col=1, line_width=0, fillcolor=le_colour, opacity=0.1)
    fig.add_vrect(x0=t_te_ts_lb, x1=t_te_ts_ub, row=4, col=1, line_width=0, fillcolor=te_colour, opacity=0.1)
    
    
fig.update_yaxes(title_text='SolO<BR>B_RTN [nT]', row=1, col=1)
fig.update_yaxes(title_text='SolO Pred<BR>B_RTN [nT]', row=2, col=1)
fig.update_yaxes(title_text='DSCOVR<BR>B_RTN [nT]', row=3, col=1)
fig.update_yaxes(title_text='DSCOVR<BR>v[km/s]', row=4, col=1)
fig.update_yaxes(title_text='STA Beacon<BR>B_RTN [nT]', row=5, col=1)

fig.update_yaxes(showgrid=True, zeroline=False, showticklabels=True,
                     showspikes=True, spikemode='across', spikesnap='cursor', showline=False, spikedash='solid', spikethickness=1)
fig.update_xaxes(showgrid=True, zeroline=False, showticklabels=True, rangeslider_visible=False,
                     showspikes=True, spikemode='across', spikesnap='cursor', showline=False, spikedash='solid', spikethickness=1)

t_now = datetime.utcnow()
fig.add_vline(x=t_now)

fig.update_layout(
    title=f'Distance SolO to Earth = {r_sep:.2f} AU (lon = {lon_recent:.2f} deg [HEEQ]) <br>Constant speed {v_cme} kms/from SolO to Earth <br>Time Delay = {t_delay:.2f} hours')

fig.write_html(f'/Users/emmadavies/Documents/ASWO/RealTime_Preparation/Realtime_Prediction_{t_now}.html')
